In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import pickle
sys.path.append(r"C:\Users\Alireza\Documents\Git\Cosine-Project")
from inSilico_experiments.utils.pothook_analysis_lib import *
from torchvision.transforms import ToTensor, ToPILImage, Pad
from torchvision.utils import make_grid
import seaborn as sns

# loading metadata and set saving root
you should set the root path of the metadata and the saving path of the figures

In [3]:
# load metadata 
data_path = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed"

#metadata_df_cosine_intralayer = pd.read_hdf(os.path.join(data_path, "metadata_df_inter_layer_with_normalized_dist.h5"), key="metadata_df")
#metadata_df_cosine_crosslayer = pd.read_hdf(os.path.join(data_path, "metadata_df_intera_layer_with_normalized_dist.h5"), key="metadata_df")
# concatenate the two dataframes
#metadata_df_cosine = pd.concat([metadata_df_cosine_intralayer, metadata_df_cosine_crosslayer], ignore_index=False)
metadata_df_cosine = pd.read_hdf(os.path.join(data_path, "metadata_df_with_sim_index_v2_final.h5"), key="metadata_df")
#metadata_df_cosine = metadata_df_cosine_most
save_root = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\results_poser"


In [ ]:
metadata_df_cosine.head(14
)

In [ ]:
metadata_df_cosine.iloc[4]

# Visualization of the results
in this section, we visualize the results of the experiments to have a better understanding of the results


## inter-layer 
1.  comparison of image rescosntruction for different pop size and sampling methods (i.e., random and most activated)

In [ ]:
# just one example
image_name = "ecoset_14"
layer_short_list = ["conv5", "conv4", "conv3", "conv2"]
similarity_metric = "cosine"
pop_size_list = [2, 8, 32, 128]
gen_rerun_id = 1
RF_treshold = 2
output_type = "target_img_RF_masked"
gen_image_type = "best_gen_imgs_RF_masked"
save_path = os.path.join(save_root, "recontruction_example")
os.makedirs(save_path, exist_ok=True)
# load the target image
target_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name,
                            layer_short=layer_short_list[0], output_type=output_type, pop_size=pop_size_list[-1])
rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short_list[0]}_{pop_size_list[-1]}.npz"))
# show the target image
fig, ax = plt.subplots()
ax.imshow(ToPILImage()(target_image))
# remove axis ticks
ax.set_xticks([])
ax.set_yticks([])
# save the figure
plt.savefig(os.path.join(save_path, f"{image_name}_target_image.png"), bbox_inches="tight", pad_inches=0)

# load the target image and crop it
target_image_croped = image_rf_crop(target_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                    int(rf_filter["sigma_x"]*RF_treshold*2)))
# show the target image
fig, ax = plt.subplots()
ax.imshow(ToPILImage()(target_image_croped))
# remove the axis
ax.axis("off")
# save the figure
plt.savefig(os.path.join(save_path, f"{image_name}_target_image_croped.png"), bbox_inches="tight", pad_inches=0)
# load the recontruction image for each layer and plot them in a grid 
# each row is a layer and each column is a population size
fig, ax = plt.subplots(len(layer_short_list), len(pop_size_list)+1, figsize=(12, 12))
for i, layer_short in enumerate(layer_short_list):
    target_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name,
                                layer_short=layer_short, output_type=output_type, pop_size=pop_size_list[-2])
    rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{pop_size_list[-2]}.npz"))
    target_image_croped = image_rf_crop(target_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                    int(rf_filter["sigma_x"]*RF_treshold*2)))
    # show the target image
    ax[i, 0].imshow(ToPILImage()(target_image_croped))
    ax[i, 0].set_title(f"target image", fontsize=17, fontweight="bold")
    # set x label
    ax[i, 0].set_ylabel(f"{layer_short}", fontsize=21, fontweight="bold")
    # remove just y axix
    ax[i, 0].set_xticks([])
    ax[i, 0].set_yticks([])
    for j, pop_size in enumerate(pop_size_list):
        try:
            recontruction_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                similarity_metric=similarity_metric)
            rcon_score = column_inquary("score", metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                similarity_metric=similarity_metric)
            # load the RF mask
            rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{pop_size}.npz"))
            # crop the recontruction image
            recontruction_image_croped = image_rf_crop(recontruction_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                                            int(rf_filter["sigma_x"]*RF_treshold*2)))
            # show the recontruction image           
            ax[i, j+1].imshow(ToPILImage()(recontruction_image_croped))
            # title with score and pop_size and bold
            ax[i, j+1].set_title(f"score: {rcon_score:.2f} ", fontsize=17, fontweight="bold")
        except:
            pass
        ax[i, j+1].axis("off")
plt.tight_layout()
# save the figure 
plt.savefig(os.path.join(save_path, f"{image_name}_recontruction.png"), bbox_inches="tight", pad_inches=0)

    

In [ ]:
# print one of the target images and the its recontruction for each layer and population size (load all regen images)
image_name_list = metadata_df_cosine["trget_imge_name"].unique()
layer_short_list = ["conv5", "conv4", "conv3", "conv2"]
similarity_metric_list = ["cosine", "MSE"]
pop_size_list = [2, 8, 32, 128]
RF_treshold = 2
pop_resampling_id = 0
smpling_type = "random"

output_type = "target_img_RF_masked"
gen_image_type = "best_gen_imgs_RF_masked"

# load the recontruction image for each layer and plot them in a grid 
# each row is a layer and each column is a population size
save_path = os.path.join(save_root, f"recontruction_images_{smpling_type}")
for similarity_metric in similarity_metric_list:
    save_path = os.path.join(save_path, similarity_metric)
    os.makedirs(save_path, exist_ok=True)
    for image_name in image_name_list:
        fig, ax = plt.subplots(len(layer_short_list), len(pop_size_list)+1, figsize=(20, 20))
        for i, layer_short in enumerate(layer_short_list):
            target_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name,
                                        layer_short=layer_short, output_type=output_type, pop_size=pop_size_list[-2],
                                        sub_pop_type="most", pop_resampling_id=pop_resampling_id)
                                        
            rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{pop_size_list[-2]}.npz"))
            target_image_croped = image_rf_crop(target_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                            int(rf_filter["sigma_x"]*RF_treshold*2)))
            # show the target image
            ax[i, 0].imshow(ToPILImage()(target_image_croped))
            ax[i, 0].set_title(f"target image", fontsize=23, fontweight="bold")
            # set x label
            ax[i, 0].set_ylabel(f"{layer_short}", fontsize=27, fontweight="bold")
            # remove just y axix
            ax[i, 0].set_xticks([])
            ax[i, 0].set_yticks([])
            for j, pop_size in enumerate(pop_size_list):
                try:
                    # load the recontruction image for each gen_rerun_id and append it to a tensor
                    rcon_score_list = []
                    for gen_rerun_id in range(5):
                        recontruction_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                            output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                            similarity_metric=similarity_metric, sub_pop_type=smpling_type,
                                                            pop_resampling_id=pop_resampling_id)

                        # load the RF mask
                        rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{pop_size}.npz"))
                        # crop the recontruction image
                        recontruction_image_croped = image_rf_crop(recontruction_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                                                        int(rf_filter["sigma_x"]*RF_treshold*2)))
                        rcon_score_list.append(column_inquary("score", metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                        output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                        similarity_metric=similarity_metric, sub_pop_type=smpling_type,
                                                        pop_resampling_id=pop_resampling_id))
                        if gen_rerun_id == 0:
                            recontruction_image_tensor = recontruction_image_croped.unsqueeze(0)
                        else:
                            recontruction_image_tensor = torch.cat((recontruction_image_tensor, recontruction_image_croped.unsqueeze(0)), dim=0)
                    rcon_score = np.mean(rcon_score_list)        
                    recontruction_image_grid = make_grid(recontruction_image_tensor, padding= 2, pad_value= 1, nrow=2)
                    # show the recontruction image           
                    ax[i, j+1].imshow(ToPILImage()(recontruction_image_grid))
                    # title with score and pop_size and bold
                    ax[i, j+1].set_title(f"score: {rcon_score:.2f} ", fontsize=23, fontweight="bold")
                except:
                    pass
                ax[i, j+1].axis("off")
        plt.tight_layout()
        # save the figure  
        plt.savefig(os.path.join(save_path, f"{image_name}_{similarity_metric}_recontruction_all_gen_{smpling_type}.png"), bbox_inches="tight", pad_inches=0)
        plt.close()

    

In [9]:
# print an example to compait reconstraction across layer for one population size
image_name_list = ['invar_1_flipped']
#image_name_list = metadata_df_cosine["trget_imge_name"].unique()
layer_short_list = ["conv5", "conv4", "conv3", "conv2"]
pop_size_list = [128]
RF_treshold = 2
pop_resampling_id = 0
smpling_type = "random"

output_type = "target_img_RF_masked"
gen_image_type = "best_gen_imgs_RF_masked"

# load the recontruction image for each layer and plot them in a grid 
# each row is a layer and each column is a population size
save_path = os.path.join(save_root, f"recontruction_images_{smpling_type}_all_images_random_selection")
similarity_metric  = "MSE"
save_path = os.path.join(save_path, similarity_metric)
os.makedirs(save_path, exist_ok=True)
for image_name in image_name_list:
    fig, ax = plt.subplots(len(layer_short_list), 2, figsize=(7, 20))
    for i, layer_short in enumerate(layer_short_list):
        target_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name,
                                    layer_short=layer_short, output_type=output_type, pop_size=16,
                                    sub_pop_type="most", pop_resampling_id=pop_resampling_id)
                                    
        rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{16}.npz"))
        target_image_croped = image_rf_crop(target_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                        int(rf_filter["sigma_x"]*RF_treshold*2)))
        # show the target image
        ax[i, 0].imshow(ToPILImage()(target_image_croped))
        # remove just y axix
        ax[i, 0].axis("off")
        pop_size = 128
        # load the recontruction image for each gen_rerun_id and append it to a tensor
        rcon_score_list = []
        for gen_rerun_id in range(5):
            recontruction_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                similarity_metric=similarity_metric, sub_pop_type=smpling_type,
                                                pop_resampling_id=pop_resampling_id)

            # load the RF mask
            rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{pop_size}.npz"))
            # crop the recontruction image
            recontruction_image_croped = image_rf_crop(recontruction_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                                            int(rf_filter["sigma_x"]*RF_treshold*2)))
            rcon_score_list.append(column_inquary("score", metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                            output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                            similarity_metric=similarity_metric, sub_pop_type=smpling_type,
                                            pop_resampling_id=pop_resampling_id))
            if gen_rerun_id == 0:
                recontruction_image_tensor = recontruction_image_croped.unsqueeze(0)
            else:
                recontruction_image_tensor = torch.cat((recontruction_image_tensor, recontruction_image_croped.unsqueeze(0)), dim=0)
        rcon_score = np.mean(rcon_score_list)        
        #recontruction_image_grid = make_grid(recontruction_image_tensor, padding= 2, pad_value= 1, nrow=2)
        #best_rec_image = recontruction_image_tensor[np.argmax(rcon_score_list)]
        best_rec_image = recontruction_image_tensor[1]
        # show the recontruction image           
        ax[i, 1].imshow(ToPILImage()(best_rec_image))
        # title with score and pop_size and bold
        ax[i, 1].axis("off")
    plt.tight_layout()
    # save the figure  
    plt.savefig(os.path.join(save_path, f"{image_name}_{similarity_metric}_recontruction_all_gen_{smpling_type}.pdf"), bbox_inches="tight", pad_inches=0, dpi=300)
    plt.savefig(os.path.join(save_path, f"{image_name}_{similarity_metric}_recontruction_all_gen_{smpling_type}.png"), bbox_inches="tight", pad_inches=0, dpi=300)
    plt.close()

In [ ]:
# print examples for compair most vs random
image_name_list = metadata_df_cosine["trget_imge_name"].unique()
layer_short_list = ["conv5432"]
pop_size_list = [16]
RF_treshold = 2
pop_resampling_num = 5
smpling_type = ["random", "most"]

output_type = "target_img_RF_masked"
gen_image_type = "best_gen_imgs_RF_masked"

# load the recontruction image for each layer and plot them in a grid 
# each row is a layer and each column is a population size
save_path = os.path.join(save_root, f"recontruction_images_moat_vs_random")
similarity_metric  = "MSE"
save_path = os.path.join(save_path, similarity_metric)
os.makedirs(save_path, exist_ok=True)
for image_name in image_name_list:
    fig, ax = plt.subplots(5, 3, figsize=(7, 17))
    layer_short = layer_short_list[0]
    target_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name,
                                layer_short='conv5', output_type=output_type, pop_size=16,
                                sub_pop_type="most", pop_resampling_id=0)
                                
    rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{'conv5'}_{16}.npz"))
    target_image_croped = image_rf_crop(target_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                    int(rf_filter["sigma_x"]*RF_treshold*2)))

    pop_size = pop_size_list[0]
    #
    rcon_score_list = []
    for gen_rerun_id in range(5):
        recontruction_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                            output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                            similarity_metric=similarity_metric, sub_pop_type='most', 
                                            pop_resampling_id=0)

        # load the RF mask
        rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{'conv5'}_{pop_size}.npz"))
        # crop the recontruction image
        recontruction_image_croped = image_rf_crop(recontruction_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                                        int(rf_filter["sigma_x"]*RF_treshold*2)))
        rcon_score_list.append(column_inquary("score", metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                        output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                        similarity_metric=similarity_metric, sub_pop_type='most',
                                        pop_resampling_id=0))
        if gen_rerun_id == 0:
            recontruction_image_tensor = recontruction_image_croped.unsqueeze(0)
        else:
            recontruction_image_tensor = torch.cat((recontruction_image_tensor, recontruction_image_croped.unsqueeze(0)), dim=0)
    rcon_score = np.mean(rcon_score_list)        
    #recontruction_image_grid = make_grid(recontruction_image_tensor, padding= 2, pad_value= 1, nrow=2)
    best_rec_image_most = recontruction_image_tensor[np.argmax(rcon_score_list)]

    for i in range(pop_resampling_num):
        rcon_score_list = []
        for gen_rerun_id in range(5):
            recontruction_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                similarity_metric=similarity_metric, sub_pop_type='random',
                                                pop_resampling_id=i)

            # load the RF mask
            rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{'conv5'}_{pop_size}.npz"))
            # crop the recontruction image
            recontruction_image_croped = image_rf_crop(recontruction_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                                            int(rf_filter["sigma_x"]*RF_treshold*2)))
            rcon_score_list.append(column_inquary("score", metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                            output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                            similarity_metric=similarity_metric, sub_pop_type='random',
                                            pop_resampling_id=i))
            if gen_rerun_id == 0:
                recontruction_image_tensor = recontruction_image_croped.unsqueeze(0)
            else:
                recontruction_image_tensor = torch.cat((recontruction_image_tensor, recontruction_image_croped.unsqueeze(0)), dim=0)
        rcon_score = np.mean(rcon_score_list)        
        #recontruction_image_grid = make_grid(recontruction_image_tensor, padding= 2, pad_value= 1, nrow=2)
        best_rec_image = recontruction_image_tensor[np.argmax(rcon_score_list)]
        # show the recontruction image           
        ax[i, 2].imshow(ToPILImage()(best_rec_image))
        ax[i, 2].axis("off")

        ax[i, 1].imshow(ToPILImage()(best_rec_image_most))
        ax[i, 1].axis("off")

        # show the target image
        ax[i, 0].imshow(ToPILImage()(target_image_croped))
        ax[i, 0].axis("off")
    ax[i, 0].set_yticks([])
    plt.tight_layout()
    # save the figure  
    plt.savefig(os.path.join(save_path, f"{image_name}_{similarity_metric}_recontruction_all_gen_{smpling_type}.pdf"), bbox_inches="tight", pad_inches=0, dpi=300)
    plt.savefig(os.path.join(save_path, f"{image_name}_{similarity_metric}_recontruction_all_gen_{smpling_type}.png"), bbox_inches="tight", pad_inches=0, dpi=300)
    plt.close()

2. Visual assessment of the rotation invariance of convolutional layers 

In [ ]:
# preaper the layout for the incvariance plot
import matplotlib as mpl
color_map_name_cosine = "jet"
pading_cmap_cosine=mpl.colormaps[color_map_name_cosine]
color_map_name_MSE = "jet"
pading_cmap_MSE=mpl.colormaps[color_map_name_MSE]

def addhock_normalization(inval, minval, maxval):
    return (inval - minval) / (maxval - minval)

cosine_min = .75
cosine_max = 1
MSE_min = -25
MSE_max = 3
image_name_list = metadata_df_cosine["trget_imge_name"].unique()
layer_short_list = ["conv5", "conv4", "conv3", "conv2"]
similarity_metric_list = ["cosine", "MSE"]
smpling_type = "most"
pop_size = 128
RF_treshold = 2
output_type = "target_img_RF_masked"
gen_image_type = "best_gen_imgs_RF_masked"
save_path_root = os.path.join(save_root, "rotation_invariance")
save_path = save_path_root
os.makedirs(save_path, exist_ok=True)
padding = 3
for image_name in image_name_list:
    # make a sub plot grid of size (3, len(layer_short_list)) the ratio of the row is 3:1:3
    fig, ax = plt.subplots(3, len(layer_short_list), figsize=(15, 15), gridspec_kw={'height_ratios': [3, 1, 3]})
    for i, layer_short in enumerate(layer_short_list):
        target_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name,
                                    layer_short=layer_short, output_type=output_type, pop_size=pop_size, sub_pop_type=smpling_type)
        rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{pop_size}.npz"))
        target_image_croped = image_rf_crop(target_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                        int(rf_filter["sigma_x"]*RF_treshold*2)))
        # show the target image
        ax[1, i].imshow(ToPILImage()(target_image_croped))
        #ax[1, i].set_ylabel(f"target image", fontsize=23, fontweight="bold")
        ax[1, i].set_ylabel(f"{layer_short}", fontsize=23, fontweight="bold")
        # remove just y axix
        ax[1, i].set_xticks([])
        ax[1, i].set_yticks([])
        # load the recontruction image for each gen_rerun_id and append it to a tensor
        j = 0
        for similarity_metric in similarity_metric_list:
            rcon_score_list = []
            for gen_rerun_id in range(5):
                recontruction_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                    output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                    similarity_metric=similarity_metric, sub_pop_type=smpling_type)
                # load the RF mask
                rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{pop_size}.npz"))
                # crop the recontruction image
                recontruction_image_croped = image_rf_crop(recontruction_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                                                int(rf_filter["sigma_x"]*RF_treshold*2)))
                score = column_inquary("score", metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                similarity_metric=similarity_metric, sub_pop_type=smpling_type)
                rcon_score_list.append(score)
                if similarity_metric == "cosine":
                    norm_score = addhock_normalization(score, cosine_min, cosine_max)
                    recontruction_image_croped = add_padding(recontruction_image_croped, int(padding+(padding/(i+1))), pading_cmap_cosine(norm_score)[:3])
                elif similarity_metric == "MSE":
                    norm_score = addhock_normalization(score, MSE_min, MSE_max)
                    recontruction_image_croped = add_padding(recontruction_image_croped, int(padding+(padding/(i+1))), pading_cmap_MSE(norm_score)[:3])
                if gen_rerun_id == 0:
                    recontruction_image_tensor = recontruction_image_croped.unsqueeze(0)
                else:
                    recontruction_image_tensor = torch.cat((recontruction_image_tensor, recontruction_image_croped.unsqueeze(0)), dim=0)
            rcon_score = np.mean(rcon_score_list)        
            recontruction_image_grid = make_grid(recontruction_image_tensor, padding= 3, pad_value= 1, nrow=2)
            # show the recontruction image           
            ax[j, i].imshow(ToPILImage()(recontruction_image_grid))
            ax[j, i].axis("off")
            j += 2
            # set tight layout
            
    plt.tight_layout()
    # save the figure 
    plt.savefig(os.path.join(save_path, f"{image_name}_invariance.png"), bbox_inches="tight", pad_inches=0)
    plt.close()


In [ ]:
# make a color map with matplotlib.colormaps( BuPu map) for a desired range
import matplotlib as mpl
color_map_name = "jet"
pading_cmap=mpl.colormaps[color_map_name]
# convet the color range to (.8, 1) range
lower, upper = -25, 3
#pading_cmap = mpl.colors.LinearSegmentedColormap.from_list("", [pading_cmap(lower), pading_cmap(upper)])
plt.imshow(np.linspace(lower, upper, 256).reshape(1, -1), cmap=pading_cmap)
# remove the axis
plt.axis("off")

cbar = plt.colorbar()
tick_font_size = 37
cbar.ax.tick_params(labelsize=tick_font_size)
plt.show()
# increas color bar tiks font size


In [ ]:
# plot imgae generation teragectory for each 

image_name_list = metadata_df_cosine["trget_imge_name"].unique()
layer_short_list = ["conv5"]
similarity_metric_list = ["cosine", "MSE"]
smpling_type = "most"
pop_size = 128

save_path_root = os.path.join(save_root, "invariance_generatoin_tragectory")

for image_name in image_name_list:
    
    save_path_image_root = os.path.join(save_path_root, image_name)
    
    for i, layer_short in enumerate(layer_short_list):
        for similarity_metric in similarity_metric_list:
            save_path = os.path.join(save_path_image_root, similarity_metric)
            os.makedirs(save_path, exist_ok=True)
            for gen_rerun_id in range(5):
                fig, ax = plt.subplots(figsize=(20, 20))
                generation_tragectory = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                    output_type="best_in_each_gen_imgs_RF_masked", pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                    similarity_metric=similarity_metric, sub_pop_type=smpling_type)
                ax.imshow(ToPILImage()(generation_tragectory))
                ax.axis("off")
                # save the figure 
                plt.savefig(os.path.join(save_path, f"{image_name}_tragectory_{gen_rerun_id}.png"), bbox_inches="tight", pad_inches=0)
                plt.close()


In [ ]:
# save some example of the rotaion invariance
# print an example to compait reconstraction across layer for one population size
image_name_list =  metadata_df_cosine["trget_imge_name"].unique()
layer_short_list = ["conv5", "conv5432"]
pop_size_list = [128]
RF_treshold = 2
pop_resampling_id = 0
smpling_type = "most"

output_type = "target_img_RF_masked"
gen_image_type = "best_gen_imgs_RF_masked"

# load the recontruction image for each layer and plot them in a grid 
# each row is a layer and each column is a population size
save_path = os.path.join(save_root, f"recontruction_images_{smpling_type}_one_example")
similarity_metric  = "MSE"
save_path = os.path.join(save_path, similarity_metric)
os.makedirs(save_path, exist_ok=True)
image_name = image_name_list[0]
fig, ax = plt.subplots(len(layer_short_list), 2, figsize=(7, 20))
for i, layer_short in enumerate(layer_short_list):
    target_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name,
                                layer_short=layer_short, output_type=output_type, pop_size=16,
                                sub_pop_type="most", pop_resampling_id=pop_resampling_id)
                                
    rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{16}.npz"))
    target_image_croped = image_rf_crop(target_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                    int(rf_filter["sigma_x"]*RF_treshold*2)))
    # show the target image
    ax[i, 0].imshow(ToPILImage()(target_image_croped))
    # remove just y axix
    ax[i, 0].set_xticks([])
    ax[i, 0].set_yticks([])
    pop_size = 128
    # load the recontruction image for each gen_rerun_id and append it to a tensor
    rcon_score_list = []
    for gen_rerun_id in range(5):
        recontruction_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                            output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                            similarity_metric=similarity_metric, sub_pop_type=smpling_type,
                                            pop_resampling_id=pop_resampling_id)

        # load the RF mask
        rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{pop_size}.npz"))
        # crop the recontruction image
        recontruction_image_croped = image_rf_crop(recontruction_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                                        int(rf_filter["sigma_x"]*RF_treshold*2)))
        rcon_score_list.append(column_inquary("score", metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                        output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                        similarity_metric=similarity_metric, sub_pop_type=smpling_type,
                                        pop_resampling_id=pop_resampling_id))
        if gen_rerun_id == 0:
            recontruction_image_tensor = recontruction_image_croped.unsqueeze(0)
        else:
            recontruction_image_tensor = torch.cat((recontruction_image_tensor, recontruction_image_croped.unsqueeze(0)), dim=0)
    rcon_score = np.mean(rcon_score_list)        
    #recontruction_image_grid = make_grid(recontruction_image_tensor, padding= 2, pad_value= 1, nrow=2)
    best_rec_image = recontruction_image_tensor[np.argmax(rcon_score_list)]
    # show the recontruction image           
    ax[i, j+1].imshow(ToPILImage()(best_rec_image))
    # title with score and pop_size and bold
    ax[i, j+1].axis("off")
plt.tight_layout()
# save the figure  
plt.savefig(os.path.join(save_path, f"{image_name}_{similarity_metric}_recontruction_all_gen_{smpling_type}.pdf"), bbox_inches="tight", pad_inches=0, dpi=300)
plt.close()

3. visual comparison of the image reconstruction of inter-layer and intra-layer

In [ ]:
# intera layer recontruction image generation
import matplotlib as mpl
import time
color_map_name_cosine = "jet"
pading_cmap_cosine=mpl.colormaps[color_map_name_cosine]
color_map_name_MSE = "jet"
pading_cmap_MSE=mpl.colormaps[color_map_name_MSE]

def addhock_normalization(inval, minval, maxval):
    return (inval - minval) / (maxval - minval)

cosine_min = .75
cosine_max = 1
MSE_min = -25
MSE_max = 3
#image_name_list = metadata_df_cosine["trget_imge_name"].unique()
image_name_list = ["invar_1_flipped", "invar_2", "imagenet_18"] 
layer_short_list = ["conv5", "conv5432", "conv53"]
similarity_metric_list = ["cosine", "MSE"]
smpling_type_list = ["most", "random"]
pop_size_list = [8, 16, 32, 64, 128, 256]
each_layer_pop_size_coeff = [1, .25, .5]
RF_treshold = 2
pop_resampling_id = 0
output_type = "target_img_RF_masked"
gen_image_type = "best_gen_imgs_RF_masked"
save_path_root = os.path.join(save_root, "inter_layer_intra_layer_comparision_fewer_popsize")
initTime = time.time()
for sampling_type in smpling_type_list:
    save_path = os.path.join(save_path_root, sampling_type)
    padding = 3
    num_of_reconstruction_repetitions = metadata_df_cosine["gen_rerun_id"].max()+1
    for similarity_metric in similarity_metric_list:
        save_path = os.path.join(save_path, similarity_metric)
        for image_name in image_name_list:
            # make a sub plot grid of size (3, len(layer_short_list)) the ratio of the row is 3:1:3
            fig, ax = plt.subplots( len(layer_short_list)+1 ,len(pop_size_list), figsize=(18, 15), gridspec_kw={'height_ratios': [1, 3, 3, 3]})
            for i, real_pop_size in enumerate(pop_size_list):
                
                layer_short = "conv5"
                target_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, pop_resampling_id=pop_resampling_id,
                                            layer_short=layer_short, output_type=output_type, pop_size=real_pop_size, sub_pop_type=sampling_type)
                rf_filter = np.load(os.path.join(data_path, "rf_filters", f"{layer_short}_{real_pop_size}.npz"))
                target_image_croped = image_rf_crop(target_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                                int(rf_filter["sigma_x"]*RF_treshold*2)))
                # show the target image
                ax[0, i].imshow(ToPILImage()(target_image_croped))
                ax[0, i].set_xticks([])
                ax[0, i].set_yticks([])
                # load the recontruction image for each gen_rerun_id and append it to a tensor
                
                for j, layer_short in enumerate(layer_short_list):
                    pop_size = int(real_pop_size*each_layer_pop_size_coeff[j])
                    rcon_score_list = []
                    for gen_rerun_id in range(num_of_reconstruction_repetitions):
                        recontruction_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                            output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                            similarity_metric=similarity_metric, sub_pop_type=sampling_type, pop_resampling_id=pop_resampling_id,)
                        # crop the recontruction image
                        recontruction_image_croped = image_rf_crop(recontruction_image, (int(rf_filter["sigma_y"]*RF_treshold*2),
                                                                                        int(rf_filter["sigma_x"]*RF_treshold*2)))
                        score = column_inquary("score", metadata_df_cosine, trget_imge_name=image_name, layer_short=layer_short,
                                                        output_type=gen_image_type, pop_size=pop_size, gen_rerun_id=gen_rerun_id,
                                                        similarity_metric=similarity_metric, sub_pop_type=sampling_type)
                        rcon_score_list.append(score)
                        if similarity_metric == "cosine":
                            norm_score = addhock_normalization(score, cosine_min, cosine_max)
                            recontruction_image_croped = add_padding(recontruction_image_croped, int(padding+(padding/(i+1))), pading_cmap_cosine(norm_score)[:3])
                        elif similarity_metric == "MSE":
                            norm_score = addhock_normalization(score, MSE_min, MSE_max)
                            recontruction_image_croped = add_padding(recontruction_image_croped, int(padding+(padding/(i+1))), pading_cmap_MSE(norm_score)[:3])
                        if gen_rerun_id == 0:
                            recontruction_image_tensor = recontruction_image_croped.unsqueeze(0)
                        else:
                            recontruction_image_tensor = torch.cat((recontruction_image_tensor, recontruction_image_croped.unsqueeze(0)), dim=0)
                    rcon_score = np.mean(rcon_score_list)        
                    recontruction_image_grid = make_grid(recontruction_image_tensor, padding= 3, pad_value= 1, nrow=2)
                    # show the recontruction image           
                    ax[j+1, i].imshow(ToPILImage()(recontruction_image_grid))
                    ax[j+1, i].axis("off")
                    # set tight layout       
            plt.tight_layout()
            # save the figure 
            os.makedirs(save_path, exist_ok=True)
            plt.savefig(os.path.join(save_path, f"{image_name}_{similarity_metric}_{sampling_type}.png"), bbox_inches="tight", pad_inches=0, dpi=300)
            plt.close()
            finishTime = time.time()
            print(f"Done {image_name}_{similarity_metric} in {finishTime-initTime} seconds")
            initTime = finishTime


# Analysis and plots 
1. analysis of recontraction sucssess in the sence of target and resconstruction images visual similarity 
    - inter-layer comparison of the reconstruction success

In [ ]:
# inter-layer sampling, similarity vs population size all in one plot

metric_name_list = ["MSE", "cosine"]
perceptual_similarity = "l2_distance"#"pixel_dist_normalized" # "LPIPS_similarity", "l2_distance", "sim_index"

expriment_meta_data_df_post_hock = metadata_df_cosine.copy()
layer_short_list = ["conv5", "conv4", "conv3", "conv2"]

# set font size
title_font_size = 40
axis_font_size = 32
tick_font_size = 25

line_color = sns.color_palette()
shade_color = sns.color_palette("pastel")
for sub_pop_type in ["random", "most"]:
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    for j, layer_short in enumerate(layer_short_list):
        for i, metric_name in enumerate(metric_name_list):

            if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
                mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            elif sub_pop_type == "most":
                mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            mean_result_index  = mean_result.index
            mean_result  = mean_result.values
            std_result = std_result.values

            if metric_name == "cosine":
                mean_result_index = mean_result_index[1:]
                mean_result = mean_result[1:]
                std_result = std_result[1:]

            # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
            ax[i].plot(mean_result_index, mean_result, '--', color=line_color[j], label=layer_short, linewidth=3)
            ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                                color=shade_color[j], alpha=0.4, label='')
            ## add grid
            ax[i].grid(True, which='both', axis='both', alpha=0.3)

            ax[i].set_xscale('log', base=2)

            ax[i].set_title(f"{metric_name}", fontsize=title_font_size, fontweight='bold')

        # get y lims for all the subplots
        y_lims = [ax[i].get_ylim() for i in range(len(ax))]
        # define the y lim that cincludes all the subplots y lims
        y_lim = [min([y_lims[i][0] for i in range(len(ax))]), max([y_lims[i][1] for i in range(len(ax))])]
        
        if perceptual_similarity == 'l2_distance':
                ax[0].set_ylabel(f"l2 distance", fontsize=axis_font_size)
        if perceptual_similarity == 'pixel_dist_normalized':
                ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
        else:
            ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
    ax[1].set_xlabel("Population size", fontsize=32)    
    # add legend to as layers names to the second subplot outside the plot        
    ax[1].legend( loc='upper right', bbox_to_anchor=(1.7, 1), fontsize=axis_font_size)
    # set the y lim for all the subplots
    for i in range(len(ax)):
        ax[i].set_ylim(y_lim)
        # make Y-tiks nicer 
        ax[i].set_yticks(np.arange(0, max(y_lim), round((max(y_lim)-min(y_lim))/4, 2)))
        ax[i].set_xlim([2**0-.3, 2**10])
        # increase the font size of the y  and x ticks
        ax[i].tick_params(axis='y', which='major', labelsize=tick_font_size)
        ax[i].tick_params(axis='x', which='major', labelsize=tick_font_size)
    # set title for the whole figure
    save_path = os.path.join(save_root, f"{perceptual_similarity}_vx_pop_size")
    os.makedirs(save_path, exist_ok=True)
    #plt.savefig(os.path.join(save_path, f"{sub_pop_type}.png"), bbox_inches="tight", pad_inches=0)

In [ ]:
# canity check for the similarity metric and target image to init image L2 distance
image_name_list = metadata_df_cosine["trget_imge_name"].unique()
layer_short = "conv5"
pop_size_list = 128
pop_resampling_id = 0

output_type = "target_img_RF_masked"

RF_path = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed\rf_filters"

init_img_path = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed\init_img\init_img.jpg"
init_img = Image.open(init_img_path)
init_img = ToTensor()(init_img)
l2_distance_list = []
for image_name in image_name_list:
    target_image = load_image_tsr(metadata_df_cosine, trget_imge_name=image_name,
                        layer_short=layer_short, output_type=output_type, pop_size=pop_size_list,
                        sub_pop_type="most", pop_resampling_id=pop_resampling_id)
    RF_map = np.load(os.path.join(RF_path, f"{layer_short}_{pop_size}.npz"))
    RF_filter = RF_map["fitmap"] > RF_map["fitmap"][int(RF_map["xo"]+(1.5*RF_map["sigma_x"])), int(RF_map["yo"]+(1.5*RF_map["sigma_y"]))]
    init_img_rf_masked =\
        (torch.from_numpy(np.absolute(RF_map["fitmap"][None,:,:])) / RF_map["fitmap"].max()) *\
                init_img
    l2_distance_list.append(l2_distance(target_image, init_img_rf_masked))
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(ToPILImage()(init_img_rf_masked))
ax.axis("off")

print(f'mean and std of target and init image L2 distance is {np.mean(l2_distance_list)} and {np.std(l2_distance_list)}')

In [ ]:
# inter-layer sampling, score vs population size all in one plot

metric_name_list = ["MSE", "cosine"]
col_name = "score" 

expriment_meta_data_df_post_hock = metadata_df_cosine.copy()
layer_short_list = ["conv5", "conv4", "conv3", "conv2"]

# set font size
title_font_size = 40
axis_font_size = 32
tick_font_size = 25

line_color = sns.color_palette()
shade_color = sns.color_palette("pastel")
for sub_pop_type in ["random", "most"]:
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    for j, layer_short in enumerate(layer_short_list):
        for i, metric_name in enumerate(metric_name_list):

            if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
                mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[col_name].mean().reset_index().groupby(
                                                                    "pop_size")[col_name].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[col_name].mean().reset_index().groupby(
                                                                    "pop_size")[col_name].sem()
            elif sub_pop_type == "most":
                mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size"])[col_name].mean().reset_index().groupby(
                                                            "pop_size")[col_name].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size"])[col_name].mean().reset_index().groupby(
                                                                    "pop_size")[col_name].sem()
            mean_result_index  = mean_result.index
            mean_result  = mean_result.values
            std_result = std_result.values

            if metric_name == "cosine":
                mean_result_index = mean_result_index[1:]
                mean_result = mean_result[1:]
                std_result = std_result[1:]

            # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
            
            ax[i].plot(mean_result_index, mean_result, '--', color=line_color[j], label=layer_short, linewidth=3)
            ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                                color=shade_color[j], alpha=0.4, label='')
            ## add grid
            ax[i].grid(True, which='both', axis='both', alpha=0.3)

            ax[i].set_xscale('log', base=2)

            ax[i].set_title(f"{metric_name}", fontsize=title_font_size, fontweight='bold')
        
    
    ax[0].set_ylabel(f"objective score", fontsize=axis_font_size)
    ax[1].set_xlabel("Population size", fontsize=32)    
    # add legend to as layers names to the second subplot outside the plot        
    ax[1].legend( loc='upper right', bbox_to_anchor=(1.7, 1), fontsize=axis_font_size)
    # set the y lim for all the subplots
    for i in range(len(ax)):
        # make tiks nicer 
        ax[i].set_xlim([2**0-.3, 2**10])
        # increase the font size of the y  and x ticks
        ax[i].tick_params(axis='y', which='major', labelsize=tick_font_size)
        ax[i].tick_params(axis='x', which='major', labelsize=tick_font_size)
    # set title for the whole figure
    save_path = os.path.join(save_root, f"score_vx_pop_size")
    os.makedirs(save_path, exist_ok=True)
    plt.savefig(os.path.join(save_path, f"{sub_pop_type}.png"), bbox_inches="tight", pad_inches=0)

In [ ]:
# inter-layer sampling, caompaire random and most seleccion for each layer
metric_name_list = ["MSE", "cosine"]
perceptual_similarity = "sim_index_l2"#"pixel_dist_normalized" # "LPIPS_similarity"

expriment_meta_data_df_post_hock = metadata_df_cosine.copy()
#layer_short_list = ["conv5", "conv4", "conv3", "conv2", "conv5432"]
layer_short_list = [ "conv5432"]
# set font size
title_font_size = 40
axis_font_size = 32
tick_font_size = 25

for j, layer_short in enumerate(layer_short_list):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    for i, metric_name in enumerate(metric_name_list):

        for sub_pop_type in ["random", "most"]: 
    # do the mean and sem by negleting np.nan values
            if sub_pop_type == "random":
                mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
                line_color = 'black'
                shade_color = 'lightgray'
                lable = "random"
            if sub_pop_type == "most":
                mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
                line_color = 'blue'
                shade_color = 'lightblue'
                lable = "most active "
            mean_result_index  = mean_result.index
            mean_result  = mean_result.values
            std_result = std_result.values

            if metric_name == "cosine":
                mean_result_index = mean_result_index[1:]
                mean_result = mean_result[1:]
                std_result = std_result[1:]
            if layer_short == "conv5432":
                 mean_result_index = mean_result_index * 4
            if layer_short == "conv53":
                 mean_result_index = mean_result_index * 2

            # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
            ax[i].plot(mean_result_index, mean_result, '--', color=line_color, label=lable, linewidth=3)
            ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                                color=shade_color, alpha=0.5, label='')
        ## add grid
        ax[i].grid(True, which='both', axis='both', alpha=0.3)

        ax[i].set_xscale('log', base=2)

        ax[i].set_title(f"{metric_name}", fontsize=title_font_size, fontweight='bold')

    # get y lims for all the subplots
    y_lims = [ax[i].get_ylim() for i in range(len(ax))]
    # define the y lim that cincludes all the subplots y lims
    y_lim = [min([y_lims[i][0] for i in range(len(ax))]), max([y_lims[i][1] for i in range(len(ax))])]
    
    if perceptual_similarity == 'l2_distance':
            ax[0].set_ylabel(f"l2 distance", fontsize=axis_font_size)
    if perceptual_similarity == 'pixel_dist_normalized':
            ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
    else:
        ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
    ax[1].set_xlabel("Population size", fontsize=32)    
    # add legend to as layers names to the second subplot outside the plot        
    ax[1].legend( loc='upper right', bbox_to_anchor=(2, 1), fontsize=axis_font_size)
    # set the y lim for all the subplots
    for i in range(len(ax)):
        ax[i].set_ylim(y_lim)
        # make Y-tiks nicer 
        #ax[i].set_yticks(np.arange(0, max(y_lim), round((max(y_lim)-min(y_lim))/4, 2)))
        ax[i].set_xlim([2**0-.3, 2**10])
        ax[i].set_yticks([-.3, 0, .3])
        # increase the font size of the y  and x ticks
        ax[i].tick_params(axis='y', which='major', labelsize=tick_font_size)
        ax[i].tick_params(axis='x', which='major', labelsize=tick_font_size)
    # set title for the whole figure
    save_path = os.path.join(save_root, f"{perceptual_similarity}_vx_pop_size_random_vs_most")
    os.makedirs(save_path, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f"{layer_short}.png"), bbox_inches="tight", pad_inches=0)
    plt.savefig(os.path.join(save_path, f"{layer_short}.pdf"), bbox_inches="tight", pad_inches=0, dpi=300)

- corss-layer comparison of the reconstruction success

In [ ]:
# intera-layer sampling, similarity vs population size for conv5432

metric_name_list = ["MSE", "cosine"]
perceptual_similarity = "sim_index_l2"#"pixel_dist_normalized" # "LPIPS_similarity", "l2_distance", "sim_index"

expriment_meta_data_df_post_hock = metadata_df_cosine.copy()
layer_short_list = ["conv5", "conv4", "conv3", "conv2"]
intrea_layer_short = "conv5432"
# set font size
title_font_size = 40
axis_font_size = 32
tick_font_size = 25
legend_font_size = 25

line_color = sns.color_palette()
shade_color = sns.color_palette("pastel")
for sub_pop_type in ["random", "most"]:
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    for i, metric_name in enumerate(metric_name_list):
        for j, layer_short in enumerate(layer_short_list):
        
            if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
                mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            elif sub_pop_type == "most":
                mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            mean_result_index  = mean_result.index
            mean_result  = mean_result.values
            std_result = std_result.values

            if metric_name == "cosine":
                mean_result_index = mean_result_index[1:]
                mean_result = mean_result[1:]
                std_result = std_result[1:]
            
            # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
            ax[i].plot(mean_result_index, mean_result, '--', color=line_color[j], label=layer_short, alpha=0.8, linewidth=3)
            ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                                color=shade_color[j], alpha=0.1, label='')
        #--- add entra layer to plot
        if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
            mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].mean()
            std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
        elif sub_pop_type == "most":
            mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                            (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                    "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                        "pop_size")[perceptual_similarity].mean()
            std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
        
        mean_result_index  = mean_result.index*4
        mean_result  = mean_result.values
        std_result = std_result.values

        if metric_name == "cosine":
            mean_result_index = mean_result_index[1:]
            mean_result = mean_result[1:]
            std_result = std_result[1:]
            
        # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
        ax[i].plot(mean_result_index, mean_result, '--', color='black', label="across-layers", alpha=.3, linewidth=3)
        ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                            color='gray', alpha=0.5, label='')
        #----
        ## add grid
        ax[i].grid(True, which='both', axis='both', alpha=0.3)

        ax[i].set_xscale('log', base=2)

        ax[i].set_title(f"{metric_name}", fontsize=title_font_size, fontweight='bold')

        # get y lims for all the subplots
        y_lims = [ax[i].get_ylim() for i in range(len(ax))]
        # define the y lim that cincludes all the subplots y lims
        y_lim = [min([y_lims[i][0] for i in range(len(ax))]), max([y_lims[i][1] for i in range(len(ax))])]
        

    if perceptual_similarity == 'l2_distance':
        ax[0].set_ylabel(f"l2 distance", fontsize=axis_font_size)
    if perceptual_similarity == 'pixel_dist_normalized':
        ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
    else:
        ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
    ax[1].set_xlabel("Population size", fontsize=32)    
    # add legend to as layers names to the second subplot outside the plot        
    ax[1].legend( loc='upper right', bbox_to_anchor=(1.8, 1), fontsize=legend_font_size)
    # set the y lim for all the subplots
    for i in range(len(ax)):
        ax[i].set_ylim(y_lim)
        # make Y-tiks nicer 
        #ax[i].set_yticks(np.arange(0, max(y_lim), round((max(y_lim)-min(y_lim))/4, 2)))
        ax[i].set_yticks([-.3, 0, .3])
        ax[i].set_xlim([2**0-.3, 2**10])
        # increase the font size of the y  and x ticks
        ax[i].tick_params(axis='y', which='major', labelsize=tick_font_size, rotation = 45)
        ax[i].tick_params(axis='x', which='major', labelsize=tick_font_size)
    save_path = os.path.join(save_root, f"{perceptual_similarity}_vx_pop_size_across_layers_recording", f"{intrea_layer_short}")
    os.makedirs(save_path, exist_ok=True)
    plt.savefig(os.path.join(save_path, f"{sub_pop_type}.pdf"), bbox_inches="tight", pad_inches=0)

In [ ]:
# intera-layer sampling, similarity vs population size for conv53

metric_name_list = ["MSE", "cosine"]
perceptual_similarity = "sim_index"#"pixel_dist_normalized" # "LPIPS_similarity"

expriment_meta_data_df_post_hock = metadata_df_cosine.copy()
layer_short_list = ["conv5", "conv3"]
intrea_layer_short = "conv53"
# set font size
title_font_size = 40
axis_font_size = 32
tick_font_size = 25
legend_font_size = 25

line_color = sns.color_palette()
shade_color = sns.color_palette("pastel")
for sub_pop_type in ["random", "most"]:
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    for i, metric_name in enumerate(metric_name_list):
        for j, layer_short in enumerate(layer_short_list):
        
            if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
                mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            elif sub_pop_type == "most":
                mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            mean_result_index  = mean_result.index
            mean_result  = mean_result.values
            std_result = std_result.values

            if metric_name == "cosine":
                mean_result_index = mean_result_index[1:]
                mean_result = mean_result[1:]
                std_result = std_result[1:]
            
            # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
            ax[i].plot(mean_result_index, mean_result, '--', color=line_color[j], label=layer_short, alpha=0.2, linewidth=3)
            ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                                color=shade_color[j], alpha=0.1, label='')
        #--- add entra layer to plot
        if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
            mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].mean()
            std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
        elif sub_pop_type == "most":
            mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                            (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                    "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                        "pop_size")[perceptual_similarity].mean()
            std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
        
        mean_result_index  = mean_result.index*2
        mean_result  = mean_result.values
        std_result = std_result.values

        if metric_name == "cosine":
            mean_result_index = mean_result_index[1:]
            mean_result = mean_result[1:]
            std_result = std_result[1:]
            
        # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
        ax[i].plot(mean_result_index, mean_result, '--', color='black', label="across-layers", alpha=.7, linewidth=3)
        ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                            color='gray', alpha=0.5, label='')
        #----
        ## add grid
        ax[i].grid(True, which='both', axis='both', alpha=0.3)

        ax[i].set_xscale('log', base=2)

        ax[i].set_title(f"{metric_name}", fontsize=title_font_size, fontweight='bold')

        # get y lims for all the subplots
        y_lims = [ax[i].get_ylim() for i in range(len(ax))]
        # define the y lim that cincludes all the subplots y lims
        y_lim = [min([y_lims[i][0] for i in range(len(ax))]), max([y_lims[i][1] for i in range(len(ax))])]
        

    if perceptual_similarity == 'l2_distance':
                ax[0].set_ylabel(f"l2 distance", fontsize=axis_font_size)
    if perceptual_similarity == 'pixel_dist_normalized':
            ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
    else:
        ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
    ax[1].set_xlabel("Population size", fontsize=32)    
    # add legend to as layers names to the second subplot outside the plot        
    ax[1].legend( loc='upper right', bbox_to_anchor=(1.8, 1), fontsize=legend_font_size)
    # set the y lim for all the subplots
    for i in range(len(ax)):
        ax[i].set_ylim(y_lim)
        # make Y-tiks nicer 
        ax[i].set_yticks(np.arange(0, max(y_lim), round((max(y_lim)-min(y_lim))/4, 2)))
        ax[i].set_xlim([2**0-.3, 2**10])
        # increase the font size of the y  and x ticks
        ax[i].tick_params(axis='y', which='major', labelsize=tick_font_size)
        ax[i].tick_params(axis='x', which='major', labelsize=tick_font_size)
    save_path = os.path.join(save_root, f"{perceptual_similarity}_vx_pop_size_across_layers_recording", f"{intrea_layer_short}")
    os.makedirs(save_path, exist_ok=True)
    plt.savefig(os.path.join(save_path, f"{sub_pop_type}.png"), bbox_inches="tight", pad_inches=0)

In [ ]:
# intera-layer sampling, score vs population size for conv5432

metric_name_list = ["MSE", "cosine"]
perceptual_similarity = "score"#"pixel_dist_normalized" # "LPIPS_similarity"

expriment_meta_data_df_post_hock = metadata_df_cosine.copy()
layer_short_list = ["conv5", "conv4", "conv3", "conv2"]
intrea_layer_short = "conv5432"
# set font size
title_font_size = 40
axis_font_size = 32
tick_font_size = 25
legend_font_size = 25

line_color = sns.color_palette()
shade_color = sns.color_palette("pastel")
for sub_pop_type in ["random", "most"]:
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    for i, metric_name in enumerate(metric_name_list):
        for j, layer_short in enumerate(layer_short_list):
        
            if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
                mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            elif sub_pop_type == "most":
                mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            mean_result_index  = mean_result.index
            mean_result  = mean_result.values
            std_result = std_result.values

            if metric_name == "cosine":
                mean_result_index = mean_result_index[1:]
                mean_result = mean_result[1:]
                std_result = std_result[1:]
            
            # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
            ax[i].plot(mean_result_index, mean_result, '--', color=line_color[j], label=layer_short, alpha=0.2, linewidth=3)
            ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                                color=shade_color[j], alpha=0.1, label='')
        #--- add entra layer to plot
        if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
            mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].mean()
            std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
        elif sub_pop_type == "most":
            mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                            (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                    "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                        "pop_size")[perceptual_similarity].mean()
            std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
        
        mean_result_index  = mean_result.index*4
        mean_result  = mean_result.values
        std_result = std_result.values

        if metric_name == "cosine":
            mean_result_index = mean_result_index[1:]
            mean_result = mean_result[1:]
            std_result = std_result[1:]
            
        # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
        ax[i].plot(mean_result_index, mean_result, '--', color='black', label="across-layers", alpha=.7, linewidth=3)
        ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                            color='gray', alpha=0.5, label='')
        #----
        ## add grid
        ax[i].grid(True, which='both', axis='both', alpha=0.3)

        ax[i].set_xscale('log', base=2)

        ax[i].set_title(f"{metric_name}", fontsize=title_font_size, fontweight='bold')

        

    if perceptual_similarity == 'l2_distance':
                ax[0].set_ylabel(f"l2 distance", fontsize=axis_font_size)
    if perceptual_similarity == 'pixel_dist_normalized':
            ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
    else:
        ax[0].set_ylabel(f"{perceptual_similarity}", fontsize=axis_font_size)
    ax[1].set_xlabel("Population size", fontsize=32)    
    # add legend to as layers names to the second subplot outside the plot        
    ax[1].legend( loc='upper right', bbox_to_anchor=(1.8, 1), fontsize=legend_font_size)
    # set the y lim for all the subplots
    for i in range(len(ax)):
        # make tiks nicer 
        ax[i].set_xlim([2**0-.3, 2**10])
        # increase the font size of the y  and x ticks
        ax[i].tick_params(axis='y', which='major', labelsize=tick_font_size)
        ax[i].tick_params(axis='x', which='major', labelsize=tick_font_size)
    save_path = os.path.join(save_root, f"{perceptual_similarity}_vx_pop_size_across_layers_recording", f"{intrea_layer_short}")
    os.makedirs(save_path, exist_ok=True)
    plt.savefig(os.path.join(save_path, f"{sub_pop_type}.png"), bbox_inches="tight", pad_inches=0)

In [ ]:
# intera-layer sampling, SCORE vs population size for conv53

metric_name_list = ["MSE", "cosine"]
perceptual_similarity = "score"#"pixel_dist_normalized" # "LPIPS_similarity"

expriment_meta_data_df_post_hock = metadata_df_cosine.copy()
layer_short_list = ["conv5", "conv3"]
intrea_layer_short = "conv53"
# set font size
title_font_size = 40
axis_font_size = 32
tick_font_size = 25
legend_font_size = 25

line_color = sns.color_palette()
shade_color = sns.color_palette("pastel")
for sub_pop_type in ["random", "most"]:
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    for i, metric_name in enumerate(metric_name_list):
        for j, layer_short in enumerate(layer_short_list):
        
            if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
                mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            elif sub_pop_type == "most":
                mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].mean()
                std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                        (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                            expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                            expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                                "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                    "pop_size")[perceptual_similarity].sem()
            mean_result_index  = mean_result.index
            mean_result  = mean_result.values
            std_result = std_result.values

            if metric_name == "cosine":
                mean_result_index = mean_result_index[1:]
                mean_result = mean_result[1:]
                std_result = std_result[1:]
            
            # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
            ax[i].plot(mean_result_index, mean_result, '--', color=line_color[j], label=layer_short, alpha=0.2, linewidth=3)
            ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                                color=shade_color[j], alpha=0.1, label='')
        #--- add entra layer to plot
        if sub_pop_type == "random":
            # do the mean and sem by negleting np.nan values
            mean_result =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].mean()
            std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
        elif sub_pop_type == "most":
            mean_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                            (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                    "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                        "pop_size")[perceptual_similarity].mean()
            std_result = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                    (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                        expriment_meta_data_df_post_hock["layer_short"]==intrea_layer_short) & (
                                                        expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                            "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                                "pop_size")[perceptual_similarity].sem()
        
        mean_result_index  = mean_result.index*2
        mean_result  = mean_result.values
        std_result = std_result.values

        if metric_name == "cosine":
            mean_result_index = mean_result_index[1:]
            mean_result = mean_result[1:]
            std_result = std_result[1:]
            
        # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
        ax[i].plot(mean_result_index, mean_result, '--', color='black', label="across-layers", alpha=.7, linewidth=3)
        ax[i].fill_between(mean_result_index, mean_result-std_result, mean_result+std_result,
                            color='gray', alpha=0.5, label='')
        #----
        ## add grid
        ax[i].grid(True, which='both', axis='both', alpha=0.3)

        ax[i].set_xscale('log', base=2)

        ax[i].set_title(f"{metric_name}", fontsize=title_font_size, fontweight='bold')


    if perceptual_similarity == 'l2_distance':
                ax[0].set_ylabel(f"l2 distance", fontsize=axis_font_size)
    if perceptual_similarity == 'pixel_dist_normalized':
            ax[0].set_ylabel(f"similarity index", fontsize=axis_font_size)
    else:
        ax[0].set_ylabel(f"{perceptual_similarity}", fontsize=axis_font_size)
    ax[1].set_xlabel("Population size", fontsize=32)    
    # add legend to as layers names to the second subplot outside the plot        
    ax[1].legend( loc='upper right', bbox_to_anchor=(1.8, 1), fontsize=legend_font_size)
    # set the y lim for all the subplots
    for i in range(len(ax)):
        # make Y-tiks nicer 
        ax[i].set_xlim([2**0-.3, 2**10])
        # increase the font size of the y  and x ticks
        ax[i].tick_params(axis='y', which='major', labelsize=tick_font_size)
        ax[i].tick_params(axis='x', which='major', labelsize=tick_font_size)
    save_path = os.path.join(save_root, f"{perceptual_similarity}_vx_pop_size_across_layers_recording", f"{intrea_layer_short}")
    os.makedirs(save_path, exist_ok=True)
    plt.savefig(os.path.join(save_path, f"{sub_pop_type}.png"), bbox_inches="tight", pad_inches=0)

In [ ]:
# cross-layer sampling, similarity vs population size
perceptual_similarity = "l2_distance" # "LPIPS_similarity"

jetring_val = 1.1
expriment_meta_data_df_post_hock = metadata_df_cosine.copy()
layer_short_list = ["conv5432", "conv53"]
num_of_layers = [4,2]
for layer_short in layer_short_list:
    i = 0
    j = 0
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    for metric_name in metric_name_list:

        sub_pop_type = "most"
        mean_result_most = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].mean()
        std_result_most = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].sem()

        mean_result_index_most = mean_result_most.index
        mean_result_most = mean_result_most.values
        std_result_most = std_result_most.values

        sub_pop_type = "random"
        # do the mean and sem by negleting np.nan values
        mean_result_random =  expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].mean()
        std_result_random  = expriment_meta_data_df_post_hock[(expriment_meta_data_df_post_hock["output_type"] == "best_gen_imgs_RF_masked") & \
                                                (expriment_meta_data_df_post_hock["similarity_metric"]==metric_name) & (
                                                    expriment_meta_data_df_post_hock["layer_short"]==layer_short) & (
                                                    expriment_meta_data_df_post_hock["sub_pop_type"] == sub_pop_type)].groupby([
                                                        "trget_imge_name", "pop_size", "pop_resampling_id"])[perceptual_similarity].mean().reset_index().groupby(
                                                            "pop_size")[perceptual_similarity].sem()
        mean_result_index_random  = mean_result_random.index
        mean_result_random  = mean_result_random.values
        std_result_random = std_result_random.values

        # plot the results using subplots and erorbar and conected dot and log 2 scale for the x axis and make the coloe of bars similar to the color of the line
        ax[i].errorbar(mean_result_index_random* num_of_layers[j], mean_result_random, yerr=std_result_random, fmt='o-', color='blue', ecolor='lightblue', elinewidth=3, capsize=0)
        ax[i].errorbar(mean_result_index_most*jetring_val* num_of_layers[j], mean_result_most, yerr=std_result_most, fmt='o-', color='black', ecolor='lightgray', elinewidth=3, capsize=0)
        ax[i].set_xscale('log', base=2)
        # increa the font size of the x and y axis labels
        ax[i].xaxis.label.set_size(17)
        ax[i].yaxis.label.set_size(17)
        ax[i].set_title(f"Similarity metric: {metric_name}", fontsize=20, fontweight='bold')
        # chabge the font size of the x and y ticks
        ax[i].tick_params(axis='both', which='major', labelsize=15)
        ax[i].set_xlim([2**0-.3, 2**10])
        i += 1
    # get y lims for all the subplots
    y_lims = [ax[i].get_ylim() for i in range(len(ax))]
    # define the y lim that cincludes all the subplots y lims
    y_lim = [min([y_lims[i][0] for i in range(len(ax))]), max([y_lims[i][1] for i in range(len(ax))])]
    
    if perceptual_similarity == 'l2_distance':
            ax[0].set_ylabel(f"l2 distance", fontsize=32)
    else:
        ax[0].set_ylabel(f"perceptual similarity \n {perceptual_similarity}", fontsize=32)
    ax[1].set_xlabel("Population size", fontsize=32)    
    # add legend to as layers names to the second subplot outside the plot        
    ax[1].legend(["random selection", "most active"], loc='upper right', bbox_to_anchor=(1.55, 1), fontsize=22)
    # set the y lim for all the subplots
    for i in range(len(ax)):
        ax[i].set_ylim(y_lim)
    # set title for the whole figure
    fig.suptitle(f"{layer_short}\n", fontsize=25, fontweight='bold', color='gray')
    save_path = os.path.join(save_root, "perceptual_similarity_vx_pop_size_acroess_layers")
    os.makedirs(save_path, exist_ok=True)
    plt.savefig(os.path.join(save_path, f"{layer_short}.png"), bbox_inches="tight", pad_inches=0)
    j += 1

- get recunstraction celling

In [25]:
def sim_index_l2(target_imag, reconstracted_imag, init_img, RFfilter=None):
    if RFfilter is not None:
        target_imag[:, np.logical_not(RFfilter)] = 0
        reconstracted_imag[:, np.logical_not(RFfilter)] = 0
        init_img[:, np.logical_not(RFfilter)] = 0
    delta_rec = (target_imag - reconstracted_imag).square().sum().sqrt()
    delta_init = (target_imag - init_img).square().sum().sqrt()
    return ((delta_init-delta_rec)/delta_init).item()

from PIL import Image
from inSilico_experiments.utils.func_lib import *
from torchvision.transforms import ToTensor

refimgdir = r"C:\Data\cosine\insilico_experiments\data\big_data_set"
inverted_imgs = r"N:\PonceLab\Users\Alireza\insilico_experiments\gan_inverted\big_data_set_moreIter"
RF_path = r"N:\PonceLab\Users\Alireza\insilico_experiments\rf_filters"
init_image_path = r"N:\PonceLab\Users\Alireza\insilico_experiments\init_imgs\fc6_init_img.jpg"

refimgnms, refimgtsr = load_ref_imgs(
        imgdir=refimgdir, preprocess_type='center_crop', image_size=227)
refimgtsr = resize_and_pad_tsr(refimgtsr, (147, 147), (40, 40))
inverted_imgnms, inverted_gtsr = load_ref_imgs(
        imgdir=inverted_imgs, preprocess_type='center_crop', image_size=227)
init_img = Image.open(init_image_path)
# convert init image to the torch tensor
inti_img_ts = ToTensor()(init_img)
net_name = "alexnet"
layer_short = "conv5"
pop_size = 128

RF_map = np.load(os.path.join(RF_path, f"{net_name}_{layer_short}_{pop_size}.npz"))

RF_filter = RF_map["fitmap"] > RF_map["fitmap"][int(RF_map["xo"]+(1.5*RF_map["sigma_x"])), int(RF_map["yo"]+(1.5*RF_map["sigma_y"]))]
init_img_rf_masked =\
    (torch.from_numpy(np.absolute(RF_map["fitmap"][None,:,:])) / RF_map["fitmap"].max()) *\
    inti_img_ts
init_img_rf_masked =\
    (torch.from_numpy(np.absolute(RF_map["fitmap"][None,:,:])) / RF_map["fitmap"].max()) *\
    inti_img_ts

diat = sim_index_l2(refimgtsr, inverted_gtsr, init_img_rf_masked, RFfilter=None)


In [26]:
diat

0.8816975185747928

In [19]:
# conctinate inverted_gtsr and refimgnms vertically
refimgnms = np.array(refimgnms)
inverted_imgnms = np.array(inverted_imgnms)
#cat_arr = np.concatenate([refimgnms, inverted_imgnms], axis=1)
refimgnms


array(['ecoset_10.JPEG', 'ecoset_11.JPEG', 'ecoset_12.JPEG',
       'ecoset_14.JPEG', 'ecoset_16.JPEG', 'ecoset_26.JPEG',
       'ecoset_31.JPEG', 'ecoset_33.JPEG', 'ecoset_49.JPEG',
       'ecoset_7.JPEG', 'imagenet_116.JPEG', 'imagenet_14.JPEG',
       'imagenet_18.JPEG', 'imagenet_26.JPEG', 'imagenet_35.JPEG',
       'imagenet_5.JPEG', 'imagenet_52.JPEG', 'imagenet_58.JPEG',
       'imagenet_7.JPEG', 'imagenet_8.JPEG', 'imagenet_83.JPEG',
       'invar_1.JPEG', 'invar_1_flipped.JPEG', 'invar_2.JPEG',
       'invar_2_rotated.JPEG', 'invar_3.JPEG', 'invar_3_flipped.JPEG',
       'invar_3_rotated.JPEG', 'invar_4.JPEG', 'invar_4_flipped.JPEG',
       'n05716342_1492.JPEG', 'n08524735_17105.JPEG'], dtype='<U20')

In [21]:
inverted_imgnms

array(['ecoset_10.jpg', 'ecoset_11.jpg', 'ecoset_12.jpg', 'ecoset_14.jpg',
       'ecoset_16.jpg', 'ecoset_26.jpg', 'ecoset_31.jpg', 'ecoset_33.jpg',
       'ecoset_49.jpg', 'ecoset_7.jpg', 'imagenet_116.jpg',
       'imagenet_14.jpg', 'imagenet_18.jpg', 'imagenet_26.jpg',
       'imagenet_35.jpg', 'imagenet_5.jpg', 'imagenet_52.jpg',
       'imagenet_58.jpg', 'imagenet_7.jpg', 'imagenet_8.jpg',
       'imagenet_83.jpg', 'invar_1.jpg', 'invar_1_flipped.jpg',
       'invar_2.jpg', 'invar_2_rotated.jpg', 'invar_3.jpg',
       'invar_3_flipped.jpg', 'invar_3_rotated.jpg', 'invar_4.jpg',
       'invar_4_flipped.jpg', 'n05716342_1492.jpg', 'n08524735_17105.jpg'],
      dtype='<U19')